# 1. Importing Libraries:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error


# 2. Loading the Dataset:

In [2]:
# Load the dataset
data = pd.read_csv('US_Regional_Sales_Data.csv')  # https://www.kaggle.com/datasets/talhabu/us-regional-sales-data
data.head(2)
data.dtypes

OrderNumber          object
Sales Channel        object
WarehouseCode        object
ProcuredDate         object
OrderDate            object
ShipDate             object
DeliveryDate         object
CurrencyCode         object
_SalesTeamID          int64
_CustomerID           int64
_StoreID              int64
_ProductID            int64
Order Quantity        int64
Discount Applied    float64
Unit Cost            object
Unit Price           object
dtype: object

# 3. Data Preprocessing:

In [3]:
# Convert date columns to datetime objects
date_columns = ['ProcuredDate', 'OrderDate', 'ShipDate', 'DeliveryDate']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], dayfirst=True)

# Remove commas and convert numerical columns to appropriate types
numeric_columns = ['Order Quantity', 'Discount Applied', 'Unit Cost', 'Unit Price']
data[numeric_columns] = data[numeric_columns].replace({',': ''}, regex=True).astype(float)

data.dtypes

OrderNumber                 object
Sales Channel               object
WarehouseCode               object
ProcuredDate        datetime64[ns]
OrderDate           datetime64[ns]
ShipDate            datetime64[ns]
DeliveryDate        datetime64[ns]
CurrencyCode                object
_SalesTeamID                 int64
_CustomerID                  int64
_StoreID                     int64
_ProductID                   int64
Order Quantity             float64
Discount Applied           float64
Unit Cost                  float64
Unit Price                 float64
dtype: object

# 4. Data Visualization:

Time Series Analysis - Sales Trends Over Time

In [ ]:
data.set_index('OrderDate', inplace=True)
data.resample('M').sum()['Unit Price'].plot()  # Monthly sales trends
plt.title('Monthly Sales Trends')
plt.ylabel('Total Sales')
plt.show()

Sales Channel Comparison - Bar Chart

In [ ]:
sns.barplot(x='Sales Channel', y='Unit Price', data=data)
plt.title('Sales Channel Comparison')
plt.ylabel('Average Unit Price')
plt.show()


Product Analysis - Pie Chart

In [ ]:
product_sales = data.groupby('_ProductID')['Unit Price'].sum()
plt.pie(product_sales, labels=product_sales.index, autopct='%1.1f%%')
plt.title('Product Sales Distribution')
plt.show()


Discount Analysis - Scatter Plot

In [ ]:
sns.scatterplot(x='Discount Applied', y='Unit Price', data=data)
plt.title('Discount Analysis')
plt.xlabel('Discount Applied')
plt.ylabel('Unit Price')
plt.show()


# 5. Feature Modeling

In [4]:
# date-based Features:

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

# Convert date columns to datetime format
data['ProcuredDate'] = pd.to_datetime(data['ProcuredDate'], dayfirst=True)
data['OrderDate'] = pd.to_datetime(data['OrderDate'], dayfirst=True)
data['ShipDate'] = pd.to_datetime(data['ShipDate'], dayfirst=True)
data['DeliveryDate'] = pd.to_datetime(data['DeliveryDate'], dayfirst=True)

# Extract day of the week, month, quarter, and year from date columns
data['ProcuredDayOfWeek'] = data['ProcuredDate'].dt.dayofweek
data['ProcuredMonth'] = data['ProcuredDate'].dt.month
data['ProcuredQuarter'] = data['ProcuredDate'].dt.quarter
data['ProcuredYear'] = data['ProcuredDate'].dt.year

data['OrderDayOfWeek'] = data['OrderDate'].dt.dayofweek
data['OrderMonth'] = data['OrderDate'].dt.month
data['OrderQuarter'] = data['OrderDate'].dt.quarter
data['OrderYear'] = data['OrderDate'].dt.year

data['ShipDayOfWeek'] = data['ShipDate'].dt.dayofweek
data['ShipMonth'] = data['ShipDate'].dt.month
data['ShipQuarter'] = data['ShipDate'].dt.quarter
data['ShipYear'] = data['ShipDate'].dt.year

data['DeliveryDayOfWeek'] = data['DeliveryDate'].dt.dayofweek
data['DeliveryMonth'] = data['DeliveryDate'].dt.month
data['DeliveryQuarter'] = data['DeliveryDate'].dt.quarter
data['DeliveryYear'] = data['DeliveryDate'].dt.year


In [5]:
# Sales Performance Metrics:

# Calculate sales performance metrics

data['Revenue'] = (data['Unit Price'] * data['Order Quantity']).astype(float)
data['Profit'] = (data['Revenue'] - (data['Unit Cost'] * data['Order Quantity'])).astype(float)
data['DiscountRatio'] = (data['Discount Applied'] / data['Unit Price']).astype(float)

# Display the updated dataset with the new metrics
print(data.head())


   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...  ShipMonth  \
0   2018-06-19          USD             6           15  ...          6   
1   2018-07-02          USD            14           20  ...          6   
2   2018-07-01          USD            21           16  ...          6   
3   2018-06-07          USD            28           48  ...          6   
4   2018-06-26          USD            22           49  ...          6   

   ShipQuarter  ShipYear  DeliveryDayOfWeek  DeliveryMonth  Delivery

In [6]:
data.dtypes

OrderNumber                  object
Sales Channel                object
WarehouseCode                object
ProcuredDate         datetime64[ns]
OrderDate            datetime64[ns]
ShipDate             datetime64[ns]
DeliveryDate         datetime64[ns]
CurrencyCode                 object
_SalesTeamID                  int64
_CustomerID                   int64
_StoreID                      int64
_ProductID                    int64
Order Quantity              float64
Discount Applied            float64
Unit Cost                   float64
Unit Price                  float64
ProcuredDayOfWeek             int64
ProcuredMonth                 int64
ProcuredQuarter               int64
ProcuredYear                  int64
OrderDayOfWeek                int64
OrderMonth                    int64
OrderQuarter                  int64
OrderYear                     int64
ShipDayOfWeek                 int64
ShipMonth                     int64
ShipQuarter                   int64
ShipYear                    

In [7]:
# Categorical Encodings:

# One-hot encode the 'Sales Channel' and 'WarehouseCode' columns
data = pd.get_dummies(data, columns=['Sales Channel', 'WarehouseCode'], drop_first=True)


In [8]:
# Time Since Previous Order:

# Sort the data by 'CustomerID' and 'OrderDate'
data.sort_values(['_CustomerID', 'OrderDate'], inplace=True)

# Calculate time since the previous order for each customer
data['TimeSincePreviousOrder'] = data.groupby('_CustomerID')['OrderDate'].diff().dt.days


In [9]:
#  Aggregated Features:

# Calculate mean and total sales for each customer
customer_sales = data.groupby('_CustomerID')['Revenue'].agg(['mean', 'sum']).reset_index()
customer_sales.rename(columns={'mean': 'AvgCustomerSales', 'sum': 'TotalCustomerSales'}, inplace=True)

# Merge aggregated features back to the original dataset
data = data.merge(customer_sales, on='_CustomerID', how='left')


In [10]:
# Interaction Features:

data['Interaction'] = data['Order Quantity'] * data['Discount Applied']
data['DiscountToCostRatio'] = data['Discount Applied'] / data['Unit Cost']
data['DiscountToQuantityRatio'] = data['Discount Applied'] / data['Order Quantity']


In [11]:
# Historical Features:

data['CumulativeRevenue'] = data.groupby(['_ProductID'])['Revenue'].cumsum()
data['CumulativeProfit'] = data.groupby(['_CustomerID'])['Profit'].cumsum()
data['CumulativeQuantity'] = data.groupby(['_ProductID', '_CustomerID'])['Order Quantity'].cumsum()


In [12]:
# Duration Feature

data['OrderToShipDuration'] = (data['ShipDate'] - data['OrderDate']).dt.days
data['OrderToDeliveryDuration'] = (data['DeliveryDate'] - data['OrderDate']).dt.days


In [13]:
# Customer Lifetime Features:

customer_first_order = data.groupby('_CustomerID')['OrderDate'].min()
customer_last_order = data.groupby('_CustomerID')['OrderDate'].max()
data['CustomerLifetime'] = (customer_last_order - customer_first_order).dt.days


# 6. Data Modeling - Price Prediction:

In [19]:
# Prepare features and target
X = data[['Order Quantity', 'Discount Applied', 'Unit Cost']]
y = data['Unit Price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

# Random Forest Regression
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Neural Network Regression
nn_model = MLPRegressor(random_state=42)
nn_model.fit(X_train, y_train)
nn_pred = nn_model.predict(X_test)

# Evaluate models
print("  Root Mean Squared Error (RMSE) ")
print("Linear Regression RMSE:", mean_squared_error(y_test, lr_pred, squared=False))
print("Random Forest Regression RMSE:", mean_squared_error(y_test, rf_pred, squared=False))
print("Neural Network Regression RMSE:", mean_squared_error(y_test, nn_pred, squared=False))
print(" ")
from sklearn.metrics import r2_score

# Calculate R-squared
r2_lr = r2_score(y_test, lr_pred)
r2_rf = r2_score(y_test, rf_pred)
r2_nn = r2_score(y_test, nn_pred)

print("   Coefficient of Determination (R-squared) ")
print("Linear Regression R-squared:", r2_lr)
print("Random Forest R-squared:", r2_rf)
print("Neural Network R-squared:", r2_nn)
print(" ")
from sklearn.metrics import mean_absolute_error

mae_lr = mean_absolute_error(y_test, lr_pred)
mae_rf = mean_absolute_error(y_test, rf_pred)
mae_nn = mean_absolute_error(y_test, nn_pred)

print("   Mean Absolute Error (MAE) ")
print("Linear Regression MAE:", mae_lr)
print("Random Forest MAE:", mae_rf)
print("Neural Network MAE:", mae_nn)
print(" ")

from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error
mse_lr = mean_squared_error(y_test, lr_pred)
mse_rf = mean_squared_error(y_test, rf_pred)
mse_nn = mean_squared_error(y_test, nn_pred)

print("  Mean Squared Error (MSE)")
print("Linear Regression MSE:", mse_lr)
print("Random Forest MSE:", mse_rf)
print("Neural Network MSE:", mse_nn)
print(" ")

# Random Forest is the best model

  Root Mean Squared Error (RMSE) 
Linear Regression RMSE: 568.1500245030115
Random Forest Regression RMSE: 562.0114244321398
Neural Network Regression RMSE: 569.6856231746407
 
   Coefficient of Determination (R-squared) 
Linear Regression R-squared: 0.8871029275211023
Random Forest R-squared: 0.8895293501633315
Neural Network R-squared: 0.8864918252685968
 
   Mean Absolute Error (MAE) 
Linear Regression MAE: 410.68697339236445
Random Forest MAE: 370.73626398890184
Neural Network MAE: 400.1836662251424
 
  Mean Squared Error (MSE)
Linear Regression MSE: 322794.45034277264
Random Forest MSE: 315856.84119224286
Neural Network MSE: 324541.7092518787
 


C:\Users\bhavi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [ ]:
# Convert 'OrderDate' to datetime
data['OrderDate'] = pd.to_datetime(data['OrderDate'], dayfirst=True)

# Set 'OrderDate' as the index
data.set_index('OrderDate', inplace=True)

# Resample and sum 'Unit Price' on a monthly basis
monthly_sales = data['Unit Price'].resample('M').sum()

# Plot the monthly sales trends
plt.figure(figsize=(10, 6))
monthly_sales.plot()
plt.title('Monthly Sales Trends')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller # determing d value

adf_result = adfuller(monthly_sales)
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])


In [ ]:
from statsmodels.tsa.stattools import acf, pacf

# Calculate autocorrelation and partial autocorrelation values
# Use a smaller value for nlags to avoid the error
nlags = 15  # Adjust this value as needed
acf_values = acf(monthly_sales, nlags=nlags)
pacf_values = pacf(monthly_sales, nlags=nlags)

# Plot ACF
plt.figure(figsize=(12, 6))
plt.subplot(211)
plt.stem(acf_values)
plt.title('Autocorrelation Function (ACF)')
plt.xlabel('Lags')
plt.ylabel('ACF')
plt.grid(True)

# Plot PACF
plt.subplot(212)
plt.stem(pacf_values)
plt.title('Partial Autocorrelation Function (PACF)')
plt.xlabel('Lags')
plt.ylabel('PACF')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Fit ARIMA model
p= 6
d= 0
q= 5
order = (p, d, q)  # Choose appropriate values for p, d, q
model = sm.tsa.ARIMA(monthly_sales, order=order)
results = model.fit()

# Forecast future values
forecast_steps = 12  # Forecast for 12 months ahead
forecast = results.get_forecast(steps=forecast_steps)

# Plot original data and forecasted values
plt.figure(figsize=(10, 6))
plt.plot(monthly_sales, label='Actual')
plt.plot(forecast.predicted_mean, label='Forecast', color='red')
plt.fill_between(forecast.conf_int().index,
                 forecast.conf_int()['lower Unit Price'],
                 forecast.conf_int()['upper Unit Price'], color='pink', alpha=0.3)
plt.title('Monthly Sales Forecast')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.legend()
plt.show()
